<h1> Desafio do kNN</h1>

Fala pessoal, como vocês estão?
Espero que todos estejam bem e saudáveis!

Gostaríamos de propor a vocês um desafio para estimular as diversas habilidades e rever os conteúdos vistos até agora no MBA. O desafio será implementar o algoritmo kNN.
As seguintes características são requeridas:
* O kNN deve ser implementado usando POO (semelhante ao do scikit-learn)
* Deve ser implementada a métrica de distância euclidiana e manhattan
* De preferência usar numpy internamente para fazer os cálculos e manipulações
* A classe kNN deve conter os seguintes métodos:
```python
fit(X_train, y_train)
predict(X_test)
__init__(self, k=3, metric="euclidean")
```

A ideia do desafio é desenvolver as várias habilidades vistas até agora no curso.
No desafio do KNN você irá desenvolver habilidades de:
* programação
* programação orientada a objeto (POO)
* manipulação de dados com numpy
* ciência de dados implementando o algoritmo kNN

É importante lembra-los de que o desafio não é obrigatório e não vale nota. Contudo será interessante para desenvolver as habilidades vistas no curso até agora. Além disso, a ideia é interagir uns com os outros, lendo código e comentando as diversas soluções que surgirão.

<b> Dicas:</b>
* comece fazendo o \__init\__
    * salve o valor de k e metric em self
* como kNN não tem um modelo, fit() apenas salvará os dados X_train e y_train enviados
* o método predict será o mais desafiador
    * comece fazendo com for e lista, sem o numpy
    * crie métodos internos para calcular as distâncias
    * depois que estiver confortável com sua versão, comece a otimiza-la utilizando o numpy

<h2> Classe KNN</h2>

Para ajuda-los, eu desenvolvi esse protótipo da classe kNN, mas fiquem a vontade para fazer as alterações que acharem necessária!

In [1]:
import numpy as np

class KNNClassifier:
    def __init__(self, k=3, metric="euclidean", p=2, verbose=False, random_state=0):
        # armazenando valores
        self.k = k
        self.metric = metric
        self.p = p
        self.verbose = verbose
        
        np.random.seed(random_state)
        
        # selecionando qual distancia usar
        self.metric = None
        if metric == "euclidean":
            self.metric_function = self.euclidean
        elif metric == "manhattan":
            self.metric_function = self.manhattan
        elif metric == "chebyshev":
            self.metric_function = self.chebyshev
        elif metric == "minkowski":
            self.metric_function = self.minkowski
        
        if verbose == True:
            print("Algoritmos kNN iniciado.")
            print("  * k      = ", k)
            print("  * metric = ", metric)
            print("  * p      = ", p, "\n")
        
    def euclidean(self, u, v):
        summ = 0
        m = u.shape[0]
        
        for i in range(m):
            summ = summ + (u[i]-v[i])**2
        distance = np.sqrt(summ)
        
        return distance
    
    def manhattan(self, u, v):
        summ = 0
        m = u.shape[0]
        
        for i in range(m):
            summ = summ + np.absolute(u[i]-v[i])
        distance = summ
        
        return distance
         
    def chebyshev(self, u, v):
        return np.max(np.absolute(u-v))
    
    def minkowski(self, u, v):
        p = self.p
        
        return np.power(np.sum(np.power(np.absolute(u-v), p)), 1.0/p)
    
    def get_k_neighbors(self, X_train, query):
        verbose = self.verbose
        k = self.k # k vizinhos
        n = X_train.shape[0] # numero de linhas
        
        # criamos vetor com n zeros
        # vamos salvar todas as distancias aqui
        distances = np.zeros(n) 
        
        for i in range(n): # para cada linha em X
            result = self.metric_function(X_train[i], query) # calcula a distancia para o ponto de consulta
            distances[i] = result
        
        # Essa função ordena as distâncias (crescente) e retorna as linhas
        row_ids = np.argsort(distances)
        
        # pegar apenas os k vizinhos
        k_neighbors_ids = row_ids[0:k]
        
        if verbose == True:
            print("Os k vizinhos são as linhas: ", k_neighbors_ids)
            print("As distancias são: ", np.round(distances[k_neighbors_ids], 2))
        
        return k_neighbors_ids
        
    def fit(self, X_train, y_train):
        verbose = self.verbose
        
        if verbose == True:
            print("Estou com preguiça, não vou gerar modelo.")
            print("Vou salvar os dados apenas!\n")

        self.X_train = X_train
        self.y_train = y_train
        
        return self
        
    
    def predict_proba(self, X_test):
        verbose = self.verbose
        if verbose == True:
            print("Vou pegar os dados que guardei para fazer a predição!\n")
            print("Vou predizer probabilidades!\n")
        
        n = X_test.shape[0] # quantidade de exemplos para predição
        X_train = self.X_train # exemplos do conjunto de treino
        y_train = self.y_train # classes do conjunto de treino
        cls_train = np.unique(y_train) # classes do treino
        n_cls_train = cls_train.shape[0] # numero de classes do treino
        y_predicted = np.zeros((n, n_cls_train), dtype="float") # matriz para colocar resultado

        
        for i in range(n):
            
            if verbose == True:
                print("Predizendo linha [", i, "]")

            # pega os ids das linhas dos k vizinhos mais proximos
            k_neighbors_ids = self.get_k_neighbors(X_train, X_test[i])

            # pega a classe dos k viznhos
            k_neighbors_classes = y_train[k_neighbors_ids]

            # verificar se houve um empate
            cls_test, cls_test_counts = np.unique(k_neighbors_classes, return_counts=True)
                        
            if verbose == True:
                print("Classes = ", cls_test)
                print("Counts = ", cls_test_counts, "\n")
            
            probs = cls_test_counts/np.sum(cls_test_counts)
            
            for j in range(cls_test.shape[0]):
                index = np.where(cls_test[j] == cls_train)[0]
                y_predicted[i][index] = probs[j]

        return y_predicted
    
    def predict(self, X_test):
        verbose = self.verbose
        if verbose == True:
            print("Vou pegar os dados que guardei para fazer a predição!\n")

        n = X_test.shape[0] # quantidade de exemplos para predição
        X_train = self.X_train # exemplos do conjunto de treino
        y_train = self.y_train # classes do conjunto de treino
        y_predicted = np.zeros(n, dtype="int") # vetor para colocar resultado
        
        for i in range(n):
            
            if verbose == True:
                print("Predizendo linha [", i, "]")


            # pega os ids das linhas dos k vizinhos mais proximos
            k_neighbors_ids = self.get_k_neighbors(X_train, X_test[i])

            # pega a classe dos k viznhos
            k_neighbors_classes = y_train[k_neighbors_ids]

            # verificar se houve um empate
            classes, counts = np.unique(k_neighbors_classes, return_counts=True)
            is_tie = np.all(counts == counts[0]) and classes.shape[0] > 1
            
            if verbose == True:
                print("Classes = ", classes)
                print("Counts = ", counts, "\n")

            if is_tie:  # se sim, chutar uma das classes
                if verbose == True:
                    print("Houve um empate!\n")
                new_class = np.random.choice(classes).reshape(1,) 

            else: # se nao, calcula a moda
                new_class = stats.mode(k_neighbors_classes)[0]
                
            y_predicted[i] = new_class

        return y_predicted

In [2]:
import numpy as np
from scipy import stats


class KNNRegressor:
    def __init__(self, k=3, metric="euclidean", p=2, verbose=False, random_state=0):
        # armazenando valores
        self.k = k
        self.metric = metric
        self.p = p
        self.verbose = verbose
        
        np.random.seed(random_state)
        
        # selecionando qual distancia usar
        self.metric = None
        if metric == "euclidean":
            self.metric_function = self.euclidean
        elif metric == "manhattan":
            self.metric_function = self.manhattan
        elif metric == "chebyshev":
            self.metric_function = self.chebyshev
        elif metric == "minkowski":
            self.metric_function = self.minkowski
        
        if verbose == True:
            print("Algoritmos kNN iniciado.")
            print("  * k      = ", k)
            print("  * metric = ", metric)
            print("  * p      = ", p, "\n")
        
    def euclidean(self, u, v):
        summ = 0
        m = u.shape[0]
        
        for i in range(m):
            summ = summ + (u[i]-v[i])**2
        distance = np.sqrt(summ)
        
        return distance
    
    def manhattan(self, u, v):
        summ = 0
        m = u.shape[0]
        
        for i in range(m):
            summ = summ + np.absolute(u[i]-v[i])
        distance = summ
        
        return distance
         
    def chebyshev(self, u, v):
        return np.max(np.absolute(u-v))
    
    def minkowski(self, u, v):
        p = self.p
        
        return np.power(np.sum(np.power(np.absolute(u-v), p)), 1.0/p)
    
    def get_k_neighbors(self, X_train, query):
        verbose = self.verbose
        k = self.k # k vizinhos
        n = X_train.shape[0] # numero de linhas
        
        # criamos vetor com n zeros
        # vamos salvar todas as distancias aqui
        distances = np.zeros(n) 
        
        for i in range(n): # para cada linha em X
            result = self.metric_function(X_train[i], query) # calcula a distancia para o ponto de consulta
            distances[i] = result
        
        # Essa função ordena as distâncias (crescente) e retorna as linhas
        row_ids = np.argsort(distances)
        
        # pegar apenas os k vizinhos
        k_neighbors_ids = row_ids[0:k]
        
        if verbose == True:
            print("Os k vizinhos são as linhas: ", k_neighbors_ids)
            print("As distancias são: ", np.round(distances[k_neighbors_ids], 2))
        
        return k_neighbors_ids
        
    def fit(self, X_train, y_train):
        verbose = self.verbose
        
        if verbose == True:
            print("Estou com preguiça, não vou gerar modelo.")
            print("Vou salvar os dados apenas!\n")

        self.X_train = X_train
        self.y_train = y_train
        
        return self
            
    def predict(self, X_test):
        verbose = self.verbose
        if verbose == True:
            print("Vou pegar os dados que guardei para fazer a predição!\n")

        n = X_test.shape[0] # quantidade de exemplos para predição
        X_train = self.X_train # exemplos do conjunto de treino
        y_train = self.y_train # valores alvo do conjunto de treino
        y_predicted = np.zeros(n, dtype="float") # vetor para colocar resultado
        
        for i in range(n):
            if verbose == True:
                print("Predizendo linha [", i, "]")

            # pega os ids das linhas dos k vizinhos mais proximos
            k_neighbors_ids = self.get_k_neighbors(X_train, X_test[i])

            # pega os valores dos k viznhos
            k_neighbors_values = y_train[k_neighbors_ids]
           
            if verbose == True:
                print("Values = ", np.round(k_neighbors_values, 2), "\n")

            new_value = np.mean(k_neighbors_values)
            y_predicted[i] = new_value

        return y_predicted

Para testar o seu código:

### Testando kNN para classificação

In [3]:
from sklearn import datasets
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

iris = datasets.load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.2, random_state = 42)

model = KNNClassifier(k=3, metric="euclidean", verbose=1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test[:2])

acc = round(accuracy_score(y_test[:2], y_pred), 2)
print(f"Accuracy: {acc}")

Algoritmos kNN iniciado.
  * k      =  3
  * metric =  euclidean
  * p      =  2 

Estou com preguiça, não vou gerar modelo.
Vou salvar os dados apenas!

Vou pegar os dados que guardei para fazer a predição!

Predizendo linha [ 0 ]
Os k vizinhos são as linhas:  [79 90 39]
As distancias são:  [0.22 0.3  0.44]
Classes =  [1]
Counts =  [3] 

Predizendo linha [ 1 ]
Os k vizinhos são as linhas:  [48 14 94]
As distancias são:  [0.33 0.39 0.47]
Classes =  [0]
Counts =  [3] 

Accuracy: 1.0


In [4]:
from sklearn import datasets
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

iris = datasets.load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.2, random_state = 42)

model = KNNClassifier(k=10, metric="euclidean", verbose=1)
model.fit(X_train, y_train)
y_pred_prob = model.predict_proba(X_test[:5])
print("proba: \n", y_pred_prob)

Algoritmos kNN iniciado.
  * k      =  10
  * metric =  euclidean
  * p      =  2 

Estou com preguiça, não vou gerar modelo.
Vou salvar os dados apenas!

Vou pegar os dados que guardei para fazer a predição!

Vou predizer probabilidades!

Predizendo linha [ 0 ]
Os k vizinhos são as linhas:  [ 79  90  39  73  80 111  92  16  62  81]
As distancias são:  [0.22 0.3  0.44 0.51 0.51 0.52 0.53 0.54 0.58 0.62]
Classes =  [1 2]
Counts =  [8 2] 

Predizendo linha [ 1 ]
Os k vizinhos são as linhas:  [ 48  14  94 114  13  41 117  84   7   1]
As distancias são:  [0.33 0.39 0.47 0.51 0.52 0.55 0.56 0.62 0.62 0.64]
Classes =  [0]
Counts =  [10] 

Predizendo linha [ 2 ]
Os k vizinhos são as linhas:  [ 24  21  64  96 106  37 119  69  19 101]
As distancias são:  [0.41 0.55 0.89 0.93 0.96 1.22 1.25 1.29 1.39 1.45]
Classes =  [2]
Counts =  [10] 

Predizendo linha [ 3 ]
Os k vizinhos são as linhas:  [ 90  79  86  39  22  42  20  81 111   6]
As distancias são:  [0.2  0.24 0.33 0.35 0.41 0.44 0.47 0.48 0.49

In [5]:
from sklearn import datasets
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

iris = datasets.load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.2, random_state = 42)

model = KNNClassifier(k=5, metric="minkowski", p=1.5)
# model = KNeighborsClassifier(n_neighbors=5, p=1.5)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

acc = round(accuracy_score(y_test, y_pred), 2)
print(f"Accuracy: {acc}")

Accuracy: 1.0


### Testando kNN para regressão

In [6]:
from sklearn import datasets
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor

boston = datasets.load_boston()
X = boston.data
y = boston.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.2, random_state = 42)

model = KNNRegressor(k=3, metric="euclidean", verbose=1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test[:2])

r2 = round(r2_score(y_test[:2], y_pred), 2)
print(f"Score: {r2}")

Algoritmos kNN iniciado.
  * k      =  3
  * metric =  euclidean
  * p      =  2 

Estou com preguiça, não vou gerar modelo.
Vou salvar os dados apenas!

Vou pegar os dados que guardei para fazer a predição!

Predizendo linha [ 0 ]
Os k vizinhos são as linhas:  [371 203 245]
As distancias são:  [10.5  10.67 10.8 ]
Values =  [21.6 23.8 29.9] 

Predizendo linha [ 1 ]
Os k vizinhos são as linhas:  [ 82 166 374]
As distancias são:  [ 6.4   9.97 18.02]
Values =  [33.1 32.  33.2] 

Score: 0.94


In [7]:
from sklearn import datasets
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor

boston = datasets.load_boston()
X = boston.data
y = boston.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.2, random_state = 42)

model = KNNRegressor(k=3, metric="euclidean")
# model = KNeighborsRegressor(n_neighbors=3, metric="euclidean")
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

r2 = round(r2_score(y_test, y_pred), 2)
print(f"Score: {r2}")

Score: 0.7
